Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Source: [Prometheus](https://prometheus.io)

- - -

[Prometheus](https://prometheus.io) is an open source system monitoring and alerting toolkit originally developed at SoundCloud.

Prometheus joined the [Cloud Native Computing Foundation](https://www.cncf.io/) in 2016 as the second hosted project after Kubernetes.

Along with two other products, the Monitoring Open Source Stack is as follows:

* **Prometheus** is a time series database and monitoring tool that is used to query metric endpoints and fetch and process the data provided by those endpoints. Data can be queried using PromQL.
* **Grafana** is a data visualization and analysis tool that allows you to create dashboards and charts for your metrics data.
* **Alertmanager**, which is usually deployed with Prometheus, forms the alert layer of the stack. It processes alerts generated by Prometheus and deduplicates (eliminates redundant data), groups and forwards them e.g. via email.


***
### Installation

The easiest way to install Prometheus is to use [helm](https://helm.io).


In [ ]:
%%bash
kubectl create namespace monitoring
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install monitoring prometheus-community/kube-prometheus-stack -n monitoring

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana)
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

are only accessible within the cluster. Therefore we have to unlock them first or open them to the outside:

In [ ]:
! kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "LoadBalancer"}}'

If you are activated, we can reach you like other services via cluster IP and port:

In [ ]:
! echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Login with user/password **admin/prom-operator**. On the left you can choose between a number of preparatory dashboards, e.g. Nodes.

In the **Prometheus** interface, resources can be selected using the query language [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/), e.g. by querying `apiserver_storage_objects`.

The **Alertmanager** is used to process alerts. See [Notifikation Beispiel](https://prometheus.io/docs/alerting/notification_examples/) for an example

- - -
### Custom resources

The Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) provides a set of [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) for monitoring.

* **Prometheus**, for a Prometheus instance.
* **Alertmanager**, for an Alertmanager instance
* **ServiceMonitor**, for monitoring Kubernetes services.
* **PodMonitor**, for monitoring pods.


In [ ]:
! kubectl api-resources | grep monitoring
! echo "---------------------------------"
! kubectl explain servicemonitors                   